In [ ]:
import os
import random
import shutil
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline


from itertools import chain
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Model, load_model
from skimage.transform import resize
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from skimage.morphology import label
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from skimage.io import imread, imshow, concatenate_images
from keras.layers.merge import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tqdm import tqdm_notebook, tnrange
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
#JSON file for validation split, create CSV for results, encode mask, IoU metric

# dictionary with the format shown in the Evaluation tab
def create_json():
    training = os.listdir(os.path.join(destination_training, 'images', 'img'))
    validation = os.listdir(os.path.join(destination_validation, 'images', 'img'))

    dataset_split = {'training': training, 'validation': validation}   
    with open('dataset_split.json', 'w') as fp:
          json.dump(dataset_split, fp)
        
        

        
        
#def dice_loss(y_true, y_pred):
 #   numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
  #  denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))
   # return 1 - numerator / denominator
    
def combinated_loss(y_true, y_pred):
    alpha = 0.5
    return alpha*dice_loss(y_true, y_pred) + (1-alpha)*tf.keras.losses.binary_crossentropy(y_true, y_pred)
    
    


#IoU Metric
def my_IoU(y_true, y_pred):
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.int32) # when using sigmoid. Use argmax for softmax

    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union


#Encode mask for CSV file
def rle_encode(img):
   
    img = np.round(np.squeeze(img)).astype(np.float32)
  
    #Flatten column-wise
    pixels = img.T.flatten()
   # pixels = pixels[:-65536]   #provo a togliere la metà dei pixel che viene tutta 1
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)





#Create CSV
def create_csv(results, results_dir='./'):
    
    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join('./', csv_fname), 'w') as f:
        f.write('ImageId,EncodedPixels,Width,Height\n')
        count = 0
        for key, value in results.items():
            #f.write(key + ',' + str(value) + '\n')
            f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')

In [ ]:
# Set some parameters
im_width = 256
im_height = 256
border = 5
#path_train = '../input/train/'
#path_test = '../input/test/'

In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = False

# Create training ImageDataGenerator object
# We need two different generators for images and corresponding masks
if apply_data_augmentation:
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                            width_shift_range=10,
                                            height_shift_range=10,
                                            zoom_range=0.3,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='constant',
                                            cval=0,
                                            rescale=1./255)
    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                             width_shift_range=10,
                                             height_shift_range=10,
                                             zoom_range=0.3,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='constant',
                                             rescale=1./255,
                                             cval=0)
else:
    train_img_data_gen = ImageDataGenerator(rescale=1./255)
    train_mask_data_gen = ImageDataGenerator(rescale=1./255)

# Create validation and test ImageDataGenerator objects
valid_img_data_gen = ImageDataGenerator(rescale=1./255)
valid_mask_data_gen = ImageDataGenerator(rescale=1./255)
test_img_data_gen = ImageDataGenerator(rescale=1./255)
test_mask_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
#Create working directory. Splitting training and validation

validation_split = 0.2

#Directories
source_dir = '/kaggle/input/ann-and-dl-image-segmentation/Segmentation_Dataset'
destination_dir = '/kaggle/working/dataset'
destination_training = os.path.join(destination_dir, 'training')
destination_validation = os.path.join(destination_dir, 'validation')
destination_test = os.path.join(destination_dir, 'test')

path_train = destination_training
path_test = destination_test


#Se nella directory di destinazione non ci sono le cartelle dataset, training, validation e test, le creo.
#Se ci sono già, le elimino e le ricreo (per rifare ogni volta il validation diverso, penso)

print("Creating main directories..")
if os.path.exists(destination_dir):
    shutil.rmtree(destination_dir)
if not os.path.exists(destination_dir):
    os.mkdir(destination_dir)
if not os.path.exists(destination_training):
    os.mkdir(destination_training)
if not os.path.exists(destination_validation):
    os.mkdir(destination_validation)
if not os.path.exists(destination_test):
    os.mkdir(destination_test)
    
#Create images/img and masks/img folder into training and validation directories. Create img into test directory

print("Creating subdirectories..")
if not os.path.exists(os.path.join(destination_training, 'images')):
    os.mkdir(os.path.join(destination_training, 'images'))
if not os.path.exists(os.path.join(destination_training, 'masks')):
    os.mkdir(os.path.join(destination_training, 'masks'))
    
if not os.path.exists(os.path.join(destination_training, 'images/img')):
    os.mkdir(os.path.join(destination_training, 'images/img'))
if not os.path.exists(os.path.join(destination_training, 'masks/img')):
    os.mkdir(os.path.join(destination_training, 'masks/img'))
    
    
if not os.path.exists(os.path.join(destination_validation, 'images')):
    os.mkdir(os.path.join(destination_validation, 'images'))
if not os.path.exists(os.path.join(destination_validation, 'masks')):
    os.mkdir(os.path.join(destination_validation, 'masks'))
    
if not os.path.exists(os.path.join(destination_validation, 'images/img')):
    os.mkdir(os.path.join(destination_validation, 'images/img'))
if not os.path.exists(os.path.join(destination_validation, 'masks/img')):
    os.mkdir(os.path.join(destination_validation, 'masks/img'))
    
if not os.path.exists(os.path.join(destination_test, 'images')):
    os.mkdir(os.path.join(destination_test, 'images'))
if not os.path.exists(os.path.join(destination_test, 'images/img')):
    os.mkdir(os.path.join(destination_test, 'images/img'))



#Split training_set e validation_set, spostandoli dalla directory di input a quella di lavoro


#Splitting training and  5936 1531-->1528
print("Splitting train from input into Training and Validation inside working directory...")

#messi due numeri divisibili per 8. 7 immagini su 7647 vengono però scartate (a random)
validation_size = 1528 #int(7647 * validation_split)
train_size = 6112 #5936 #7467 - validation_size

files = os.listdir(source_dir + '/training/images/img')
random.shuffle(files)
files = iter(files)
i=0

while i < validation_size:
    file=str(next(files))
    result = shutil.copy(source_dir + '/training/images/img' + '/' + file, destination_validation + '/images/img')
    result = shutil.copy(source_dir + '/training/masks/img' + '/' + file, destination_validation + '/masks/img')
    i += 1

while i < validation_size + train_size:
    file=str(next(files))
    result = shutil.copy(source_dir + '/training/images/img' + '/' + file, destination_training + '/images/img')
    result = shutil.copy(source_dir + '/training/masks/img' + '/' + file, destination_training + '/masks/img')
    i += 1
    

        

#Copio i file del test dalla directory di input a quella di lavoro

print("Copying test set from input inside working directory...")
for file in os.listdir(source_dir + '/test/images/img'):
        result = shutil.copy(source_dir + '/test/images/img' + '/' + file, destination_test + '/images/img')
       
print("creating JSON file..")
create_json()

print("Done!")

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------

# Batch size
bs = 4

# img shape
img_h = 256
img_w = 256

num_classes=2

# Training
# Two different generators for images and masks
# ATTENTION: here the seed is important!! We have to give the same SEED to both the generator
# to apply the same transformations/shuffling to images and corresponding masks

train_img_gen = train_img_data_gen.flow_from_directory(os.path.join(destination_training, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=True,
                                                       interpolation='bilinear',
                                                       seed=SEED)  
train_mask_gen = train_mask_data_gen.flow_from_directory(os.path.join(destination_training, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs,
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=True,
                                                         interpolation='bilinear',
                                                         color_mode='grayscale',
                                                         seed=SEED)
train_gen = zip(train_img_gen, train_mask_gen)  #Iteratore che mappa l'i-esimo elemento di train_img_gen con l i-esimo di train_mask_gen

# Validation
valid_img_gen = valid_img_data_gen.flow_from_directory(os.path.join(destination_validation, 'images'),
                                                       target_size=(img_h, img_w),
                                                       batch_size=bs, 
                                                       class_mode=None, # Because we have no class subfolders in this case
                                                       shuffle=False,
                                                       interpolation='bilinear',
                                                       seed=SEED)
valid_mask_gen = valid_mask_data_gen.flow_from_directory(os.path.join(destination_validation, 'masks'),
                                                         target_size=(img_h, img_w),
                                                         batch_size=bs, 
                                                         class_mode=None, # Because we have no class subfolders in this case
                                                         shuffle=False,
                                                         interpolation='bilinear',
                                                         color_mode='grayscale',
                                                         seed=SEED)
valid_gen = zip(valid_img_gen, valid_mask_gen)


In [ ]:
# Create Dataset objects
# ----------------------

# Training
# --------
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               
                                               #1: Batch_size  2:height  3:width  4:channels
                                               output_shapes=([bs, img_h, img_w, 3], [bs, img_h, img_w, 1]))
def prepare_target(x_, y_):
    y_ = tf.cast(y_, tf.int32)
    return x_, y_



#def prepare_target(x_, y_):
 #   y_ = tf.cast(tf.expand_dims(y_[..., 0], -1), tf.int32)
  #  return x_, tf.where(y_ > 0, y_ - 1, y_ + 1)

#train_dataset = train_dataset.map(prepare_target)
# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([bs, img_h, img_w, 3], [bs, img_h, img_w, 1]))
#valid_dataset = valid_dataset.map(prepare_target)

# Repeat
valid_dataset = valid_dataset.repeat()

print(tf.keras.__version__)
print(tf.__version__)

**RETE UNET N°1** **NOT USED**

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True, activ = True, dropout= False):
    # first layer
    x = tf.keras.layers.Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor) #input_tensor
    if batchnorm:
        x = tf.keras.layers.BatchNormalization()(x)
    if activ:
        x = tf.keras.layers.Activation("relu")(x)
    if dropout:
        x = tf.keras.layers.Dropout(0.2)(x)
    # second layer
    x = tf.keras.layers.Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    #if batchnorm:
        #x = tf.keras.layers.BatchNormalization()(x)
   # x = tf.keras.layers.Activation("relu")(x)
    return x



def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    inputs = tf.keras.layers.Input((img_h, img_w, 3))
    s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)
    
    c1 = conv2d_block(s, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = tf.keras.layers.MaxPooling2D((2, 2)) (c1)
    #p1 = tf.keras.layers.Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = tf.keras.layers.MaxPooling2D((2, 2)) (c2)
    #p2 = tf.keras.layers.Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = tf.keras.layers.MaxPooling2D((2, 2)) (c3)
   # p3 = tf.keras.layers.Dropout(dropout)(p3)
    
  
    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2)) (c4)
   # p4 = tf.keras.layers.Dropout(dropout)(p4)
  
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    print (n_filters)
    u6 = tf.keras.layers.Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
   # u6 = tf.keras.layers.Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = tf.keras.layers.Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    #u7 = tf.keras.layers.Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = tf.keras.layers.Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    #u8 = tf.keras.layers.Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=False, activ=False)

    u9 = tf.keras.layers.Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    #u9 = tf.keras.layers.Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    c9 = tf.keras.layers.BatchNormalization()(c9)
    c9 = tf.keras.layers.Activation("relu")(c9)
    
    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model

**RETE UNET N°2**

In [ ]:
# Build U-Net model

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
    
    
def get_unet2():
    inputs = tf.keras.layers.Input((img_h, img_w, 3))
    s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

    c1 = tf.keras.layers.Conv2D(16, (3, 3), kernel_initializer='he_normal',
                                padding='same')(s)
    c1 = tf.keras.layers.BatchNormalization()(c1)
    c1 = tf.keras.layers.Activation("relu")(c1)
    #c1 = tf.keras.layers.Dropout(0.1)(c1)
    
    c1 = tf.keras.layers.Conv2D(16, (3, 3), kernel_initializer='he_normal',
                                padding='same')(c1)
    #c1 = tf.keras.layers.BatchNormalization()(c1)
    #c1 = tf.keras.layers.Activation("relu")(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal',
                                padding='same')(p1)
    c2 = tf.keras.layers.BatchNormalization()(c2)
    c2 = tf.keras.layers.Activation("relu")(c2)
    #c2 = tf.keras.layers.Dropout(0.1)(c2)
    
    
    c2 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal',
                                padding='same')(c2)
    #c2 = tf.keras.layers.BatchNormalization()(c2)
    #c2 = tf.keras.layers.Activation("relu")(c2)
    
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal',
                                padding='same')(p2)
    c3 = tf.keras.layers.BatchNormalization()(c3)
    c3 = tf.keras.layers.Activation("relu")(c3)
    #c3 = tf.keras.layers.Dropout(0.2)(c3)
    
    c3 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal',
                                padding='same')(c3)
    #c3 = tf.keras.layers.BatchNormalization()(c3)
    #c3 = tf.keras.layers.Activation("relu")(c3)
    
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal',
                                padding='same')(p3)
    c4 = tf.keras.layers.BatchNormalization()(c4)
    c4 = tf.keras.layers.Activation("relu")(c4)
    #c4 = tf.keras.layers.Dropout(0.2)(c4)
    
    c4 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal',
                                padding='same')(c4)
    #c4 = tf.keras.layers.BatchNormalization()(c4)
    #c4 = tf.keras.layers.Activation("relu")(c4)
    
    p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal',
                                padding='same')(p4)    
    c5 = tf.keras.layers.BatchNormalization()(c5)
    c5 = tf.keras.layers.Activation("relu")(c5)
    #c5 = tf.keras.layers.Dropout(0.3)(c5)
    
    c5 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal',
                                padding='same')(c5)    
    #c5 = tf.keras.layers.BatchNormalization()(c5)
    #c5 = tf.keras.layers.Activation("relu")(c5)

    u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal',
                                padding='same')(u6)
    c6 = tf.keras.layers.BatchNormalization()(c6)
    c6 = tf.keras.layers.Activation("relu")(c6)
    #c6 = tf.keras.layers.Dropout(0.2)(c6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal',
                                padding='same')(c6)
    #c6 = tf.keras.layers.BatchNormalization()(c6)
    #c6 = tf.keras.layers.Activation("relu")(c6)

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal',
                                padding='same')(u7)
    c7 = tf.keras.layers.BatchNormalization()(c7)
    c7 = tf.keras.layers.Activation("relu")(c7)
    #c7 = tf.keras.layers.Dropout(0.2)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal',
                                padding='same')(c7)    
    #c7 = tf.keras.layers.BatchNormalization()(c7)
    #c7 = tf.keras.layers.Activation("relu")(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal',
                                padding='same')(u8)
    #c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal',
                                padding='same')(c8)    
    #c8 = tf.keras.layers.BatchNormalization()(c8)
    #c8 = tf.keras.layers.Activation("relu")(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), kernel_initializer='he_normal',
                                padding='same')(u9)  
    c9 = tf.keras.layers.BatchNormalization()(c9)
    c9 = tf.keras.layers.Activation("relu")(c9)
    c9 = tf.keras.layers.Dropout(0.2)(c9)
    
    c9 = tf.keras.layers.Conv2D(16, (3, 3), kernel_initializer='he_normal',
                                padding='same')(c9)       
    c9 = tf.keras.layers.BatchNormalization()(c9)
    c9 = tf.keras.layers.Activation("relu")(c9)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model

In [ ]:
#input_img = Input((im_height, im_width, 1), name='img')
model = get_unet2()
metrics = [my_IoU]

model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-tgs-salt.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

#results = model.fit(x = train_dataset, batch_size=4, epochs=30, callbacks=callbacks,
                    #validation_data=valid_dataset, validation_steps=len(valid_img_gen))



model.fit(x = train_dataset,
          epochs = 30,
          steps_per_epoch=len(train_img_gen), 
          validation_data=valid_dataset, 
          validation_steps=len(valid_img_gen),
          callbacks=callbacks)

**Compute Prediction**

In [ ]:
import time
from datetime import datetime
import matplotlib.pyplot as plt

from PIL import Image

%matplotlib notebook

# Cycle over test images

test_img_dir = os.path.join(destination_test, 'images', 'img') #test_dir

img_filenames = next(os.walk(test_img_dir))[2]


results = {}
i=0
for img_filename in img_filenames:
    
    i += 1
    mask_filename = img_filename[:-4] + '.tif' #'.png'
    
    img = Image.open(os.path.join(test_img_dir, img_filename))
    img = img.convert('RGB')
    img = img.resize((256, 256))
  
    
    
    img_arr = np.array(img)
    
    img_arr = np.expand_dims(np.array(img), 0)
    
    pred = model.predict(x=img_arr / 255., batch_size = 8)
   
    pred= (pred >0.5).astype(np.uint8)
    

    # Get predicted class as the index corresponding to the maximum value in the vector probability
    
    #predicted_class = tf.argmax(out_softmax, -1)
    mask = pred[0]
  
    
    
    mask_pred = rle_encode(mask)
    mask_name = os.path.splitext(img_filename)[0]
    results[mask_name] = mask_pred   
    #if i >4: 
      #  break
    
    #time.sleep(1000)
create_csv(results)